This notebook is given for information purposes. Some lines may need to be adapted for the code to run correctly.

In [ ]:
import numpy as np
import scipy.io
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)

# Load from MSEAS PPLS results

In [ ]:
dirname = '../../data/ls_output/reykjavik_dublin/'
#dirname = '/home/bastien/Documents/work/MSEAS_Path_Planning_LS/results/unprocessed/20240710_130812'
phi_fpaths = [os.path.join(dirname, fname) for fname in sorted(os.listdir(dirname)) if fname.startswith('sol_checkpoint')]
phi_frame_0 = scipy.io.loadmat(phi_fpaths[0])['phi_to_save']
phi = np.zeros((len(phi_fpaths), *phi_frame_0.shape))
for it, fpath in enumerate(phi_fpaths):
    phi[it] = scipy.io.loadmat(fpath)['phi_to_save']

In [ ]:
ff = np.load('../../data/reykjavik_dublin/ff_non_dim.npz')

In [ ]:
spatial_grid_ff = np.stack(
    np.meshgrid(
        np.linspace(*ff['bounds'][1], ff['values'].shape[1]),
        np.linspace(*ff['bounds'][2], ff['values'].shape[2]), 
        indexing='ij'), 
    -1)

time_grid_ff = np.linspace(*ff['bounds'][0], ff['values'].shape[0])

spatial_grid_phi = np.stack(
    np.meshgrid(
        np.linspace(*ff['bounds'][1], 2*ff['values'].shape[1]),
        np.linspace(*ff['bounds'][2], 2*ff['values'].shape[2]), 
        indexing='ij'), 
    -1)
print(spatial_grid_phi.shape)

In [ ]:
time_grid_ff_trunc = time_grid_ff[:13]
space_time_grid_trunc = np.stack(
    np.meshgrid(
        time_grid_ff_trunc,
        np.linspace(*ff['bounds'][1], ff['values'].shape[1]),
        np.linspace(*ff['bounds'][2], ff['values'].shape[2]), 
        indexing='ij'), 
    -1)

In [ ]:
ff['bounds']

In [ ]:
from scipy.interpolate import RegularGridInterpolator
times = np.linspace(*ff['bounds'][0], ff['values'].shape[0])
new_times = times[:13]
ff_interp = RegularGridInterpolator(
    (
        time_grid_ff_trunc,
        np.linspace(*ff['bounds'][1], ff['values'].shape[1]),
        np.linspace(*ff['bounds'][2], ff['values'].shape[2])
    ),
    ff['values'][:13, ...],
    method='cubic',
    bounds_error=False,
    fill_value=None
)

In [ ]:
d_ff = np.stack((ff_interp(space_time_grid_trunc, nu=(0, 1, 0)), ff_interp(space_time_grid_trunc, nu=(0, 0, 1))), -1)

In [ ]:
L = np.max(np.linalg.norm(d_ff, axis=(-2, -1)))
print(L)

In [ ]:
ff_interp.values.max()

In [ ]:
np.linalg.norm(d_ff, axis=(-2, -1)).shape

In [ ]:
L_mean = np.mean(np.linalg.norm(d_ff, axis=(-2, -1)))
print(L_mean)

In [ ]:
epsilon = 2.25e-2
print(f'{2 * np.pi * 1 / epsilon * np.exp(L * 1.2):.2E}')

In [ ]:
print(f'{2 * np.pi * 1 / epsilon * np.exp(L_mean * 1.2)}')

## Draw wind

In [ ]:
it_lo, it_hi = 0, 7
norm_wind = mpl.colors.Normalize()
norm_wind.autoscale([0, np.linalg.norm(ff['values'][it_lo:it_hi], axis=-1).max()])

In [ ]:
np.linalg.norm(ff['values'][it_lo:it_hi], axis=-1).max()

In [ ]:
np.linalg.norm(ff['values'][5], axis=-1).max()

In [ ]:
bbox_llcrn = np.array((0, 0))
bbox_ulcrn = np.array((0, 1))
bbox_urcrn = np.array((1.5, 1))
bbox_lrcrn = np.array((1.5, 0))

for it in range(it_lo, it_hi):
    fig, ax = plt.subplots()
    nsub = 1
    X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
    U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]
    ax.pcolormesh(X, Y, np.sqrt(U**2 + V**2), cmap='turbo', shading='gouraud', norm=norm_wind)
    n_points = 50
    lbound = np.vstack((np.linspace(bbox_llcrn[0], bbox_ulcrn[0], n_points), 
                        np.linspace(bbox_llcrn[1], bbox_ulcrn[1], n_points))).T
    
    ubound = np.vstack((np.linspace(bbox_ulcrn[0], bbox_urcrn[0], n_points), 
                        np.linspace(bbox_ulcrn[1], bbox_urcrn[1], n_points))).T
    
    rbound = np.vstack((np.linspace(bbox_urcrn[0], bbox_lrcrn[0], n_points), 
                        np.linspace(bbox_urcrn[1], bbox_lrcrn[1], n_points))).T
    
    bbound = np.vstack((np.linspace(bbox_lrcrn[0], bbox_llcrn[0], n_points), 
                        np.linspace(bbox_lrcrn[1], bbox_llcrn[1], n_points))).T
    
    boundary_vectors = np.vstack((lbound, ubound, rbound, bbound))
    white_mask = mpl.patches.Polygon(boundary_vectors, color='white', alpha=0.2, zorder=3)
    ax.add_patch(white_mask)
    nsub = 5
    X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
    U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]
    ax.quiver(X, Y, U, V)

    ax.scatter(1.25, 0.5, s=100, color='black')
    ax.scatter(0.25, 0.5, s=150, color='black', marker='*')
    
    title = f't={time_grid_ff[it]:.3g}'
    ax.set_title(title)

    plt.colorbar(mpl.cm.ScalarMappable(norm=norm_wind, cmap='turbo'), ax=ax,
    label="Wind norm [no dim.]", location='right', fraction=0.038, pad=0.04) 
    
    
    #fig.savefig(f'reykjavik_dublin/wind_{it:0>3}.pdf', format='pdf')
    #plt.close(fig)

In [ ]:
scale_l = 1.4859e6

In [ ]:
bbox_llcrn = np.array((0, 0))
bbox_ulcrn = np.array((0, 1))
bbox_urcrn = np.array((1.5, 1))
bbox_lrcrn = np.array((1.5, 0))

for it in range(it_lo, it_hi):
    nsub = 1
    X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
    U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]
    
    fig = plt.figure()
    oblique_merc_proj = ccrs.ObliqueMercator(central_longitude=-21.86, central_latitude=64.13, 
                                             false_northing=0.5, false_easting=0.25, azimuth=-113, scale_factor=1/scale_l)
    ax = fig.add_subplot(1, 1, 1, projection=oblique_merc_proj)
    ax.set_extent([0, 1.49, 0, 1], oblique_merc_proj)
    ax.coastlines(zorder=5)
    cmesh = ax.pcolormesh(X, Y, 
                  np.sqrt(U**2 + V**2), cmap='turbo', shading='gouraud', transform=oblique_merc_proj)
    
    bbox_llcrn = np.array((0, 0))
    bbox_ulcrn = np.array((0, 1))
    bbox_urcrn = np.array((1.5, 1))
    bbox_lrcrn = np.array((1.5, 0))
    n_points = 50
    lbound = np.vstack((np.linspace(bbox_llcrn[0], bbox_ulcrn[0], n_points), 
                        np.linspace(bbox_llcrn[1], bbox_ulcrn[1], n_points))).T
    
    ubound = np.vstack((np.linspace(bbox_ulcrn[0], bbox_urcrn[0], n_points), 
                        np.linspace(bbox_ulcrn[1], bbox_urcrn[1], n_points))).T
    
    rbound = np.vstack((np.linspace(bbox_urcrn[0], bbox_lrcrn[0], n_points), 
                        np.linspace(bbox_urcrn[1], bbox_lrcrn[1], n_points))).T
    
    bbound = np.vstack((np.linspace(bbox_lrcrn[0], bbox_llcrn[0], n_points), 
                        np.linspace(bbox_lrcrn[1], bbox_llcrn[1], n_points))).T
    
    boundary_vectors = np.vstack((lbound, ubound, rbound, bbound))
    white_mask = mpl.patches.Polygon(boundary_vectors, color='white', alpha=0.2, transform=oblique_merc_proj)
    ax.add_patch(white_mask)
    nsub = 5
    X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
    U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]
    ax.quiver(X, Y, U, V, transform=oblique_merc_proj)
    ax.plot(np.linspace(0.25, 1.25, 50), np.linspace(0.5, 0.5, 50), color='black', 
        transform=oblique_merc_proj, linewidth=2)
    ax.plot(np.linspace(0.25, 1.25, 50), np.linspace(0.5, 0.5, 50), color='white', 
        transform=oblique_merc_proj, linewidth=2, dashes=(4,4))
    ax.scatter(1.25, 0.5, s=100, color='black', zorder=5, edgecolor='white', transform=oblique_merc_proj)
    ax.scatter(0.25, 0.5, s=150, color='black', marker='*', zorder=5, edgecolor='white', transform=oblique_merc_proj)
    ax.add_feature(cfeature.LAND, color='lightgrey')
    
    title = f't={time_grid_ff[it]:.3g}'
    ax.set_title(title)
    fig.tight_layout()
    plt.colorbar(mpl.cm.ScalarMappable(norm=norm_wind, cmap='turbo'), ax=ax,
    label="Wind norm [no dim.]", location='right', fraction=0.032, pad=0.04)
    
    ax.gridlines(color='black')
    ax.set_xticks(np.arange(0, 1.51, 0.3))
    ax.set_yticks(np.arange(0, 1.01, 0.2))
    
    #fig.savefig(f'reykjavik_dublin/wind_{it:0>3}.pdf', format='pdf', bbox_inches='tight')
    #plt.close(fig)
    plt.show()

In [ ]:
ff_dim = np.load('../../data/reykjavik_dublin/ff.npz')

In [ ]:
ff_dim['bounds']

In [ ]:
norm_wind_dim = mpl.colors.Normalize()
norm_wind_dim.autoscale([0, np.linalg.norm(ff_dim['values'][it_lo:it_hi], axis=-1).max()])

In [ ]:
it = 0
nsub = 1
X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
U, V = ff_dim['values'][it, ::nsub, ::nsub, 0], ff_dim['values'][it, ::nsub, ::nsub, 1]

fig = plt.figure()
oblique_merc_proj = ccrs.ObliqueMercator(central_longitude=-21.86, central_latitude=64.13, azimuth=-113)
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-36.5, 6, 47, 72], ccrs.PlateCarree())
gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree(), x_inline=False, y_inline=False, rotate_labels=False)
gl.top_labels = False
gl.right_labels = False
ax.coastlines(zorder=5)
cmesh = ax.pcolormesh(scale_l * (X -0.25), scale_l * (Y - 0.5), 
              np.sqrt(U**2 + V**2), cmap='turbo', shading='gouraud', transform=oblique_merc_proj, 
                      norm=norm_wind_dim)

bbox_llcrn = (np.array((0, 0)) - np.array((0.25, 0.5))) * scale_l
bbox_ulcrn = (np.array((0, 1)) - np.array((0.25, 0.5))) * scale_l
bbox_urcrn = (np.array((1.5, 1)) - np.array((0.25, 0.5))) * scale_l
bbox_lrcrn = (np.array((1.5, 0)) - np.array((0.25, 0.5))) * scale_l
n_points = 50
lbound = np.vstack((np.linspace(bbox_llcrn[0], bbox_ulcrn[0], n_points), 
                    np.linspace(bbox_llcrn[1], bbox_ulcrn[1], n_points))).T

ubound = np.vstack((np.linspace(bbox_ulcrn[0], bbox_urcrn[0], n_points), 
                    np.linspace(bbox_ulcrn[1], bbox_urcrn[1], n_points))).T

rbound = np.vstack((np.linspace(bbox_urcrn[0], bbox_lrcrn[0], n_points), 
                    np.linspace(bbox_urcrn[1], bbox_lrcrn[1], n_points))).T

bbound = np.vstack((np.linspace(bbox_lrcrn[0], bbox_llcrn[0], n_points), 
                    np.linspace(bbox_lrcrn[1], bbox_llcrn[1], n_points))).T

boundary_vectors = np.vstack((lbound, ubound, rbound, bbound))
white_mask = mpl.patches.Polygon(boundary_vectors, color='white', alpha=0.2, transform=oblique_merc_proj)
ax.add_patch(white_mask)
nsub = 5
X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
U, V = ff_dim['values'][it, ::nsub, ::nsub, 0], ff_dim['values'][it, ::nsub, ::nsub, 1]
ax.quiver(scale_l * (X -0.25), scale_l * (Y - 0.5), U, V, transform=oblique_merc_proj)
ax.plot(scale_l * np.linspace(0, 1, 50), scale_l * np.linspace(0, 0, 50), color='black', 
        transform=oblique_merc_proj, linewidth=2)
ax.plot(scale_l * np.linspace(0, 1, 50), scale_l * np.linspace(0, 0, 50), color='white', 
        transform=oblique_merc_proj, linewidth=2, dashes=(4,4))
ax.scatter(*(scale_l * np.array((1, 0))), s=100, color='black', zorder=5, edgecolor='white', transform=oblique_merc_proj)
ax.scatter(*(scale_l * np.array((0, 0))), s=150, color='black', marker='*', zorder=5, edgecolor='white', transform=oblique_merc_proj)
ax.add_feature(cfeature.LAND, color='lightgrey')

ax.set_title('2021-11-01 05:00Z', fontsize=22)
fig.tight_layout()
plt.colorbar(mpl.cm.ScalarMappable(norm=norm_wind_dim, cmap='turbo'), ax=ax,
label="Wind norm [m/s]", location='right', fraction=0.028, pad=0.04) 

#fig.savefig('reykjavik_dublin/situation.pdf', bbox_inches='tight')

plt.show()

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl

it = 0
nsub = 3
X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]

fig = plt.figure()
oblique_merc_proj = ccrs.ObliqueMercator(central_longitude=-21.86, central_latitude=64.13, azimuth=-113, scale_factor=1/scale_l)
ax = fig.add_subplot(1, 1, 1, projection=oblique_merc_proj)
ax.set_extent([-0.25, 1.23, -0.5, 0.5], oblique_merc_proj)
ax.coastlines(zorder=5)
cmesh = ax.pcolormesh(X -0.25, Y - 0.5, 
              np.sqrt(U**2 + V**2), cmap='turbo', shading='gouraud', transform=oblique_merc_proj)

bbox_llcrn = np.array((0, 0)) - np.array((0.25, 0.5))
bbox_ulcrn = np.array((0, 1)) - np.array((0.25, 0.5))
bbox_urcrn = np.array((1.5, 1)) - np.array((0.25, 0.5))
bbox_lrcrn = np.array((1.5, 0)) - np.array((0.25, 0.5))
n_points = 50
lbound = np.vstack((np.linspace(bbox_llcrn[0], bbox_ulcrn[0], n_points), 
                    np.linspace(bbox_llcrn[1], bbox_ulcrn[1], n_points))).T

ubound = np.vstack((np.linspace(bbox_ulcrn[0], bbox_urcrn[0], n_points), 
                    np.linspace(bbox_ulcrn[1], bbox_urcrn[1], n_points))).T

rbound = np.vstack((np.linspace(bbox_urcrn[0], bbox_lrcrn[0], n_points), 
                    np.linspace(bbox_urcrn[1], bbox_lrcrn[1], n_points))).T

bbound = np.vstack((np.linspace(bbox_lrcrn[0], bbox_llcrn[0], n_points), 
                    np.linspace(bbox_lrcrn[1], bbox_llcrn[1], n_points))).T

boundary_vectors = np.vstack((lbound, ubound, rbound, bbound))
white_mask = mpl.patches.Polygon(boundary_vectors, color='white', alpha=0.2, transform=oblique_merc_proj)
ax.add_patch(white_mask)
nsub = 5
X, Y = spatial_grid_ff[::nsub, ::nsub, 0], spatial_grid_ff[::nsub, ::nsub, 1]
U, V = ff['values'][it, ::nsub, ::nsub, 0], ff['values'][it, ::nsub, ::nsub, 1]
ax.quiver(X -0.25, Y - 0.5, U, V, transform=oblique_merc_proj)
ax.plot(np.linspace(0, 1, 50), np.linspace(0, 0, 50), color='red', 
        transform=oblique_merc_proj, linewidth=2)
ax.scatter(1, 0, s=100, color='black', zorder=5, transform=oblique_merc_proj)
ax.scatter(0, 0, s=150, color='black', marker='*', zorder=5, transform=oblique_merc_proj)
ax.add_feature(cfeature.LAND, color='lightgrey')

ax.set_title('2021-11-01 05:00Z', fontsize=22)
fig.tight_layout()
plt.colorbar(mpl.cm.ScalarMappable(norm=norm_wind, cmap='turbo'), ax=ax,
label="Wind norm [m/s]", location='right', fraction=0.032, pad=0.04)

ax.gridlines(color='black')
ax.set_xticks(np.arange(-0.2, 1.22, 0.2))
ax.set_yticks(np.arange(-0.5, 0.51, 0.25))

#fig.savefig('reykjavik_dublin/situation.png', dpi=300, bbox_inches='tight')

plt.show()

# Run computation

In [ ]:
from dabry.problem import NavigationProblem
from dabry.solver_display import display
from dabry.solver_ef import SolverEFResampling
from dabry.flowfield import DiscreteFF
import pickle

In [ ]:
ff = DiscreteFF.from_npz('../../data/reykjavik_dublin/ff.npz')
x_init = np.diag((5 / 6, 1 / 2)) @ (ff.bounds[1:, 1] - ff.bounds[1:, 0])
x_target = np.diag((1 / 6, 1 / 2)) @ (ff.bounds[1:, 1] - ff.bounds[1:, 0])
pb_unscaled = NavigationProblem(ff, x_init, x_target, 23, name='reykjavik_dublin_cavi_error_short')
pb = pb_unscaled.rescale()
pb.io.case_dir = os.path.join('reykjavik_dublin', pb_unscaled.name)
print(pb.io.case_dir)

In [ ]:
solver_in_depth = SolverEFResampling(pb, free_max_step=False, rel_max_step=0.05,
                            mode_resamp_interp=False, mode_obs_stop=True)

solver_in_depth.solve()
solver_in_depth.save_results()

traj_ortho = pb.orthodromic()
traj_htarget = pb.radial()
pb.io.save_traj(traj_ortho, name='orthodromic')
pb.io.save_traj(traj_htarget, name='radial')

# fig, _ = display(solver, theme_dark=False, no_3d=True)
# fig.show()

In [ ]:
solver_in_depth_short = SolverEFResampling(pb, free_max_step=False, rel_max_step=0.001,
                            mode_resamp_interp=False, mode_obs_stop=True, max_depth=5)

solver_in_depth_short.solve()
solver_in_depth_short.save_results()

traj_ortho = pb.orthodromic()
traj_htarget = pb.radial()
pb.io.save_traj(traj_ortho, name='orthodromic')
pb.io.save_traj(traj_htarget, name='radial')

# fig, _ = display(solver, theme_dark=False, no_3d=True)
# fig.show()

In [ ]:
np.linalg.norm(solver_in_depth_short.sites['AA'].costate_init)

# Display results

In [ ]:
cost_map = np.load('/home/bastien/Documents/work/dabry/output/reykjavik_dublin_interp/cost_map.npy')
#cost_map = np.load('reykjavik_dublin/reykjavik_dublin_interp/cost_map.npy')
#cost_map = np.load('reykjavik_dublin/reykjavik_dublin_cavi_error/cost_map.npy')

In [ ]:
spatial_grid_cm = np.stack(np.meshgrid(np.linspace(*ff['bounds'][1], cost_map.shape[0]),
                           np.linspace(*ff['bounds'][2], cost_map.shape[1]), indexing='ij'), -1)

In [ ]:
trajs = []
dirname = '/home/bastien/Documents/work/dabry/output/reykjavik_dublin_interp/trajs/ef_01'
#dirname = 'reykjavik_dublin/reykjavik_dublin_interp/trajs/ef_01'

trajs_fpath = [os.path.join(dirname, fname) for fname in os.listdir(dirname) if fname.endswith('.npz')]
for fpath in trajs_fpath:
    trajs.append(np.load(fpath))

In [ ]:
fig, ax = plt.subplots()

# for traj in tqdm(trajs):
#     ax.plot(*traj['states'].T, color='grey', alpha=0.2)

# checkpoints = []
# for site in solver.sites.values():
#     checkpoints.append(site.states[site.index_t_check_next])
# checkpoints = np.array(checkpoints)
# ax.scatter(*checkpoints.T, s=1, zorder=6, color='black', alpha=0.8)

for site in tqdm(solver.sites.values()):
    ax.plot(*site.states.T, color='grey', alpha=0.2)
    
traj_ls = np.genfromtxt('/home/bastien/Documents/work/MSEAS_Path_Planning_LS/results/reykjavik_dublin/traj_opt.csv', delimiter=',')

ax.plot(*traj_ls.T, color='white', zorder=6, linewidth=3)
ax.plot(*traj_ls.T, color='blue', zorder=6)

ax.plot(*solver.solution_site.states[:87].T, color='white', zorder=6, linewidth=3)
ax.plot(*solver.solution_site.states[:87].T, color='red', zorder=6)

ax.contourf(spatial_grid_cm[..., 0], spatial_grid_cm[..., 1], cost_map, levels=np.arange(0, 1.41, 0.1), 
            cmap='jet', alpha=0.7, zorder=4)

for it in range(phi.shape[0]):
    ax.contour(spatial_grid_phi[..., 0], spatial_grid_phi[..., 1], phi[it], levels=[0], colors=['black'], linestyles='--', zorder=5)

ax.scatter(1.25, 0.5, s=100, color='black', zorder=6)
ax.scatter(0.25, 0.5, s=150, color='black', marker='*', zorder=6)

fig.savefig('reykjavik_dublin/reykjavik_dublin_interp', dpi=300)
plt.show()

In [ ]:
traj_ls

In [ ]:
fig, ax = plt.subplots()

# for traj in tqdm(trajs):
#     ax.plot(*traj['states'].T, color='grey', alpha=0.2)

checkpoints = []
validity_index = 30
max_dist = 0
site_max_dist = None
for site in tqdm(solver_in_depth.sites.values()):
    if site.index_t_check_next >= validity_index:
        state_1 = site.states[site.index_t_check_next]
        state_21 = site.next_nb[site.index_t_check_next].states[site.index_t_check_next]
        try:
            site_par = solver_in_depth.sites[solver_in_depth.site_mngr.parents_name_from_name(site.name)[1]]
            state_22 = site_par.states[site.index_t_check_next]
        except KeyError:
            state_22 = np.nan * np.ones(2)
        state_2 = state_21 if not np.any(np.isnan(state_21)) else state_22
        dist = np.linalg.norm(state_1 - state_2)
        if dist >= solver_in_depth.max_dist:
            if dist > max_dist:
                max_dist = dist
                site_max_dist = site
            checkpoints.append(site.states[site.index_t_check_next])
            # plt.plot(*np.vstack((state_1, state_2)).T, zorder=6)

# checkpoints = np.array(checkpoints)
# ax.scatter(*checkpoints.T, s=1, zorder=6, color='#443563', marker='x')
    

for site in tqdm(solver_in_depth.sites.values()):
    ax.plot(*site.states.T, color='grey', alpha=0.4, zorder=3)

ax.plot(*site_max_dist.states.T, color='yellow', zorder=3)
site_max_dist_prev = solver_in_depth.sites[solver_in_depth.site_mngr.parents_name_from_name(site_max_dist.name)[0]]
site_max_dist_next = solver_in_depth.sites[solver_in_depth.site_mngr.parents_name_from_name(site_max_dist.name)[1]]
ax.plot(*site_max_dist_prev.states.T, color='blue', zorder=3)
ax.plot(*site_max_dist_next.states.T, color='red', zorder=3)

#ax.plot(*solver_in_depth.nemesis.states.T, color='red', alpha=0.2)
#ax.plot(*solver_in_depth.nemesis.next_nb[solver_in_depth.nemesis.index_t_check_next].states.T, color='green', alpha=0.2)

# ax.contourf(spatial_grid_cm[..., 0], spatial_grid_cm[..., 1], cost_map_no_guarantee, levels=np.arange(0, 1.41, 0.1), 
#             cmap='jet', alpha=0.7, zorder=4)
#ax.contourf(spatial_grid_cm[..., 0], spatial_grid_cm[..., 1], cost_map_no_guarantee, levels=[0, 0.48], hatches=['/'], alpha=0.7, zorder=4)

# for it in range(phi.shape[0]):
#     ax.contour(spatial_grid_phi[..., 0], spatial_grid_phi[..., 1], phi[it], levels=[0], colors=['black'], linestyles='--', zorder=5)

ax.scatter(1.25, 0.5, s=100, color='black', zorder=6)
ax.scatter(0.25, 0.5, s=150, color='black', marker='*', zorder=6)
# delta_y = 0.1
# delta_x = delta_y * 1.5
# ax.set_xlim(1.25 - delta_x / 2, 1.25 + delta_x / 2)
# ax.set_ylim(0.6, 0.6 + delta_y)
# ax.set_xlim(1.2, 1.25)
# ax.set_ylim(0.65, 0.7)
fig.savefig('reykjavik_dublin/reykjavik_dublin_in_depth_divergence.png', dpi=300)
plt.show()

In [ ]:
np.linalg.norm(site_max_dist_prev.states[1] - site_max_dist_next.states[1])

In [ ]:
np.linalg.norm(site_max_dist_prev.costate_init - site_max_dist_next.costate_init)

In [ ]:
site_max_dist.states[1]

In [ ]:
print(site_max_dist.costate_init, site_max_dist_par.costate_init)

In [ ]:
solver_in_depth.pb.target_radius

In [ ]:
validity_index

In [ ]:
nemesis = solver_in_depth.nemesis
nemesis_nb = solver_in_depth.nemesis.next_nb[solver_in_depth.nemesis.index_t_check_next]
print(np.linalg.norm(nemesis.states[nemesis.index_t_check_next] - nemesis_nb.states[nemesis.index_t_check_next]))

In [ ]:
nemesis_par = solver_in_depth.sites[solver_in_depth.site_mngr.parents_name_from_name(nemesis.name)[1]]

In [ ]:
print(np.linalg.norm(nemesis.states[nemesis.index_t_check_next] - nemesis_par.states[nemesis.index_t_check_next]))

In [ ]:
cost_map_no_guarantee = solver_in_depth.get_cost_map_no_guarantee()

In [ ]:
pb_unscaled = NavigationProblem.from_name('reykjavik_dublin')
pb_unscaled.name = 'reykjavik_dublin_interp'
pb = pb_unscaled.rescale()
pb.io.case_dir = os.path.join('reykjavik_dublin', pb_unscaled.name)
print(pb.io.case_dir)

In [ ]:
solver = SolverEFResampling(pb, free_max_step=False, rel_max_step=0.05,
                            mode_resamp_interp=True, mode_obs_stop=True)

solver.solve()

solver.save_results()
with open(os.path.join(pb.io.case_dir, 'solver.pickle'), 'wb') as f:
    pickle.dump(solver, f)

traj_ortho = pb.orthodromic()
traj_htarget = pb.radial()
pb.io.save_traj(traj_ortho, name='orthodromic')
pb.io.save_traj(traj_htarget, name='radial')

# fig, _ = display(solver, theme_dark=False, no_3d=True)
# fig.show()